# Charts

In [ ]:
import numpy as np
import pandas as pd
from siuba import *
from calitp import *
import intake
import data_prep
import shared_utils
import altair as alt
import altair_saver
from shared_utils import altair_utils 
pd.options.display.max_rows = 100
pd.options.display.float_format = "{:.0f}".format

In [ ]:
BC_GTFS_NTD2 = pd.read_parquet('BC_GTFS_NTD.parquet')

In [ ]:
#Grabbing only one row for each agency.
Aggregate1 = BC_GTFS_NTD2.groupby(['organization_name','reporter_type','GTFS', 'fleet_size', 'Is_Agency_In_BC_Only_1_means_Yes']).agg({'total_vehicles':'max',
       'average_age_of_fleet__in_years_':'max','doors_sum':'max','_0_9':'max', '_10_12':'max', '_13_15':'max', '_16_20':'max', '_21_25':'max',
       '_26_30':'max', '_31_60':'max', '_60plus':'max',})

In [ ]:
#Reset index
Aggregate1 = Aggregate1.reset_index()


#  GTFS Status Charts
* One for vehicles over 9
* One for vehicles over 15

In [ ]:
vehicles_gtfs = data_prep.aggregation_one(Aggregate1, 'GTFS')
vehicles_gtfs

### Looking at vehicles 9+ by GTFS
![pic](./chart_outputs/bar_GTFS_by_vehicles_older_than_9.png)

In [ ]:
data_prep.basic_bar_chart2(vehicles_gtfs,'GTFS','vehicles_older_than_9') 

In [ ]:
data_prep.basic_bar_chart2(vehicles_gtfs,'GTFS','vehicles_percent_older_than_9') 

### Looking at vehicles 15+ by GTFS
![pic](./chart_outputs/bar_GTFS_by_vehicles_older_than_15.png)

In [ ]:
data_prep.basic_bar_chart2(vehicles_gtfs,'GTFS','vehicles_older_than_15') 

# Fleet size 

In [ ]:
fleet = data_prep.aggregation_one(Aggregate1,'fleet_size') 

In [ ]:
# don't need no info
fleet =  fleet.loc[fleet['fleet_size'] != 'No Info'] 

### Vehicles older than 9
![pic](./chart_outputs/bar_fleet_size_by_vehicles_older_than_9.png)

In [ ]:
data_prep.basic_bar_chart2(fleet,'fleet_size','vehicles_older_than_9') 

### Vehicles older than 15
![pic](./chart_outputs/bar_fleet_size_by_vehicles_older_than_15.png)

In [ ]:
data_prep.basic_bar_chart2(fleet,'fleet_size','vehicles_older_than_15') 

# Other
## Looking at organizations & GTFS
![pic](./chart_outputs/bar_GTFS_by_count_of_agencies.png)

In [ ]:
Orgs = Aggregate1.groupby(['GTFS']).agg({'organization_name':'nunique'})
Orgs = Orgs.reset_index()
Orgs = Orgs.rename(columns = {'organization_name': 'count_of_agencies'}) 
data_prep.basic_bar_chart2(Orgs,'GTFS','count_of_agencies') 

## Looking at reporter type & GTFS
![testing](./chart_outputs/bar_GTFS_by_count_of_agencies.png)

In [ ]:
Reporter_type_agg = Aggregate1.groupby(['reporter_type']).agg({'organization_name':'nunique'})
Reporter_type_agg = Reporter_type_agg.reset_index()
Reporter_type_agg = Reporter_type_agg.rename(columns = {'organization_name':'Count of Agencies'})
data_prep.basic_bar_chart2(Reporter_type_agg,'reporter_type','Count of Agencies') 

## Looking at agencies that spent the most..
![pic](./chart_outputs/bar_organization_name_by_allocationamount.png)

In [ ]:
Most_Money = BC_GTFS_NTD2.groupby(['organization_name','GTFS']).agg({'allocationamount':'sum'})

In [ ]:
Most_Money= Most_Money.sort_values('allocationamount').tail(10)
Most_Money = Most_Money.reset_index()

In [ ]:
data_prep.basic_bar_chart2(Most_Money,'organization_name','allocationamount') 